In [3]:
import sys
sys.path.append('../')

import os
import re
from datetime import datetime, timedelta

from pyspark.sql import SparkSession
from delta.pip_utils import configure_spark_with_delta_pip

from utils.string_utils import camel_to_snake, normalize_string
from engine import SupermetricsAPI
from etl import default, get_df

In [2]:
def sm_normalize_cols(s):
    s = normalize_string(s)
    s = camel_to_snake(s)
    s = s.replace(' ', '_')
    s = re.sub(r'_+', '_', s)
    s = s.strip()
    return s

In [3]:
builder = SparkSession.builder \
    .appName('test_delta_utils') \
    .config('spark.sql.warehouse.dir', 'pyspark_tables') \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.databricks.delta.retentionDurationCheck.enabled', False) \
    .config('spark.databricks.delta.schema.autoMerge.enabled', True) \
    .config('spark.databricks.delta.checkLatestSchemaOnRead', True) \
    .config("spark.log.level", "ERROR") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config('delta.enableChangeDataFeed', True)

spark = configure_spark_with_delta_pip(builder).enableHiveSupport().getOrCreate()
spark.sql("CREATE DATABASE IF NOT EXISTS supermetrics")

TB = 'supermetrics.test1'
URL = os.environ['URL']
PK = ['Sheet', 'Column A']
PARTITION = ['Sheet']
DT_START = datetime(2022, 1, 1)
DT_END = datetime(2022, 3, 1)
DT_DELTA = timedelta(days=15)
SM_KWARGS = {'offset': 100000}
R_KWARGS = {'timeout': 600}

23/04/23 20:12:22 WARN Utils: Your hostname, spiriel resolves to a loopback address: 127.0.1.1; using 192.168.15.6 instead (on interface enp3s0)
23/04/23 20:12:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ahow/main_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ahow/.ivy2/cache
The jars for the packages stored in: /home/ahow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d19a7efd-4593-4da1-b862-2e267004e8e2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 90ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   | 

23/04/23 20:12:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# df = get_df(api, spark, dt_start, dt_end, r_kwargs=r_kwargs, replace_f=[sm_normalize_cols])
# df.show()

api = SupermetricsAPI(URL, **SM_KWARGS)

default(
    api, spark, TB, PK, PARTITION,
    DT_START, DT_END, DT_DELTA,
    r_kwargs=R_KWARGS, replace_f=[sm_normalize_cols]
)

**************************************
*********** Starting SM ETL **********
**************************************
from 01/01/2022 to 15/01/2022
extracting


/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


merging
23/04/23 20:12:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.




from 16/01/2022 to 30/01/2022
extracting


/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


merging


from 31/01/2022 to 14/02/2022
extracting


/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


merging


from 15/02/2022 to 28/02/2022
extracting


/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


merging




In [6]:
spark.sql(f"""
SELECT * FROM {TB}
""").toPandas()

,sheet,column_a,column_b
0,Sheet1,1,a
1,Sheet1,2,b
2,Sheet1,3,c
3,Sheet1,4,d
4,Sheet1,5,e
5,Sheet1,6,f
6,Sheet1,7,g
7,Sheet1,8,h
8,sheet2,2,b
9,sheet2,3,c


In [7]:
spark.sql(f"""
DESCRIBE HISTORY {TB}
""").toPandas()

/home/ahow/main_env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,5,2023-04-23 20:12:57.412,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,4.0,Serializable,False,"{'numOutputRows': '15', 'numTargetRowsInserted...",None,Apache-Spark/3.3.1 Delta-Lake/2.2.0
1,4,2023-04-23 20:12:54.206,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,3.0,Serializable,False,"{'numOutputRows': '15', 'numTargetRowsInserted...",None,Apache-Spark/3.3.1 Delta-Lake/2.2.0
2,3,2023-04-23 20:12:50.953,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,2.0,Serializable,False,"{'numOutputRows': '15', 'numTargetRowsInserted...",None,Apache-Spark/3.3.1 Delta-Lake/2.2.0
3,2,2023-04-23 20:12:47.596,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,1.0,Serializable,False,"{'numOutputRows': '15', 'numTargetRowsInserted...",None,Apache-Spark/3.3.1 Delta-Lake/2.2.0
4,1,2023-04-18 01:21:18.863,None,None,MERGE,"{'matchedPredicates': '[{""actionType"":""update""...",None,None,None,0.0,Serializable,False,"{'numOutputRows': '14', 'numTargetRowsInserted...",None,Apache-Spark/3.3.1 Delta-Lake/2.2.0
5,0,2023-04-18 01:11:13.483,None,None,CREATE OR REPLACE TABLE AS SELECT,"{'description': None, 'partitionBy': '[""sheet""...",None,None,None,NaN,Serializable,False,"{'numOutputRows': '14', 'numOutputBytes': '820...",None,Apache-Spark/3.3.1 Delta-Lake/2.2.0
